<div dir=ltr align=center>
    <font color=0F5298 size=7>Neurosymbolic VQA Program Generator</font><br>
    <br>
    <font color=32CD32 size=5>Part 3: In-Context Learning (ICL)</font><br>
</div>

<br/>

---

## **Goal: In-Context Learning with LLMs**

Our final strategy is completely different. Instead of training a model *from scratch*, we'll leverage a large, pre-trained Large Language Model (LLM).

We will use **In-Context Learning (ICL)**, which means we *prompt* the model with a few examples ("shots") of a question and its corresponding program. The LLM is expected to recognize the pattern and generate a correct program for a new, unseen question without any gradient updates or fine-tuning.

**Example Prompt ($k=1$ shot):**
```json
You are an AI assistant... (system prompt)
Question: Are there any rubber spheres?
Program: <START> scene filter_shape[sphere] filter_material[rubber] exist <END>
Question: How many large blue things are there?
```

We will evaluate the LLM's performance by varying the number of shots ($k$) provided in the prompt.

## Step 1: **Setup and Dependencies**

This notebook requires the `transformers`, `torch`, and `accelerate` libraries. We also import all the necessary evaluation functions from `src.evaluation.eval_icl` and our project's config and utils.

In [ ]:
import sys
import os
import json
import torch
from transformers import pipeline
import matplotlib.pyplot as plt

# Add the project root to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import all our project modules
import src.config as config
import src.evaluation.eval_icl as icl_eval
from src.executor import ClevrExecutor
from src.vocabulary import load_vocab

# Set device
DEVICE = 0 if torch.cuda.is_available() else -1 # 0 for cuda:0, -1 for cpu
print(f"Using device: {'cuda' if DEVICE == 0 else 'cpu'}")

## Step 2: **Load Model and Data**

We need to load:
1.  The LLM pipeline from Hugging Face.
2.  The raw `train_questions.json` (to sample few-shot examples from).
3.  The raw `test_questions.json` (to evaluate on).
4.  Our `ClevrExecutor` and `vocab` (for executor-based evaluation).

In [ ]:
# 1. Load LLM Pipeline
print(f"Loading model: {config.LLM_MODEL_ID}")
pipe = pipeline(
    "text-generation",
    model=config.LLM_MODEL_ID,
    torch_dtype=torch.bfloat16,
    device=DEVICE, 
)

# 2. Load Raw Train Questions (for few-shot examples)
with open(config.TRAIN_QUESTIONS_JSON, 'r') as f:
    train_questions = json.load(f)['questions']

# 3. Load Raw Test Questions (for evaluation)
with open(config.TEST_QUESTIONS_JSON, 'r') as f:
    test_questions = json.load(f)['questions']

# 4. Load Executor and Vocab
vocab = load_vocab(config.VOCAB_JSON_FILE)
executor = ClevrExecutor(
    train_scene_json=config.TRAIN_SCENES_JSON,
    val_scene_json=config.TEST_SCENES_JSON, # Use test scenes for eval
    vocab_json=config.VOCAB_JSON_FILE
)

print("\n--- Setup Complete ---")
print(f"Loaded {len(train_questions)} train questions for sampling.")
print(f"Loaded {len(test_questions)} test questions for evaluation.")

## Step 3: **Test Few-Shot Example Generation**

Let's see what the `get_few_shot_examples` function does. It should randomly pick 2 examples and format them nicely.

In [ ]:
few_shot_context = icl_eval.get_few_shot_examples(train_questions, num_examples=2)
print(few_shot_context)

## Step 4: **Test Single LLM Generation**

Now let's combine the prompt and a test question to see what the LLM generates. We'll also test our parser.

In [ ]:
# Use 5 shots for this test
k_shots = 5
base_prompt = ( "You are an AI assistant. You must translate natural language "
                  "questions into a structured sequence of program functions. "
                  "The program must start with <START> and end with <END>.")
few_shot_context = icl_eval.get_few_shot_examples(train_questions, k_shots)
system_prompt = f"{base_prompt}\n\n{few_shot_context}"

# Get a test question
test_question_data = test_questions[0]
user_question = f"Question: {test_question_data['question']}"

print("--- System Prompt (Truncated) ---")
print(system_prompt[:500] + "...")
print("\n--- User Question ---")
print(user_question)

# --- Generate --- 
llm_output = icl_eval.generate_program_with_llm(pipe, system_prompt, user_question)
print("\n--- LLM Raw Output ---")
print(llm_output)

# --- Parse ---
parsed_program = icl_eval.parse_program_from_llm_output(llm_output)
print("\n--- Parsed Program ---")
print(parsed_program)

# --- Ground Truth ---
from src.utils.program_utils import list_to_str, list_to_prefix
gt_program_str = list_to_str(list_to_prefix(test_question_data['program']))
print("\n--- Ground Truth Program (Prefix) ---")
print(gt_program_str)

## Step 5: **Run Full Evaluation**

Now we'll loop through our list of $k$ shots (`[0, 2, 5, 10]`) and run a full evaluation on a subset of the test data (`num_test_samples`).

We will run two types of evaluation:

1.  **Executor Accuracy**: We execute the LLM's program and check if the *final answer* matches the ground-truth answer. (Measures semantic correctness).
2.  **BLEU Score**: We compare the *token sequence* of the LLM's program to the ground-truth program. (Measures syntactic similarity).

In [ ]:
shots_list = config.ICL_SHOTS_LIST
num_samples = config.ICL_NUM_TEST_SAMPLES

print("--- 1. Running Executor-based Accuracy Evaluation ---")
executor_results = icl_eval.evaluate_icl_executor(
    pipe=pipe,
    train_questions=train_questions,
    test_questions=test_questions,
    executor=executor,
    vocab=vocab,
    num_shots_list=shots_list,
    num_test_samples=num_samples,
    split='val' # Use 'val' split for executor (since we loaded test scenes as val)
)
print("\n--- Executor Evaluation Complete ---")
print(executor_results)

In [ ]:
print("--- 2. Running BLEU Score Evaluation ---")
bleu_results = icl_eval.evaluate_icl_bleu(
    pipe=pipe,
    train_questions=train_questions,
    test_questions=test_questions,
    num_shots_list=shots_list,
    num_test_samples=num_samples
)
print("\n--- BLEU Evaluation Complete ---")
print(bleu_results)

## Step 6: **Plot Final Results**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle('In-Context Learning (ICL) Performance vs. Number of Shots')

# --- Executor Accuracy Plot ---
ax1.set_title("Executor-Based Accuracy")
ax1.set_xlabel("Number of Shots in Prompt (k)")
ax1.set_ylabel("Program Execution Accuracy (%)")
if executor_results:
    shots = list(executor_results.keys())
    accuracies = [v * 100 for v in executor_results.values()]
    ax1.plot(shots, accuracies, marker='o', color='b')
    ax1.set_xticks(shots)
ax1.grid(True, linestyle='--', alpha=0.6)

# --- BLEU Score Plot ---
ax2.set_title("BLEU Score Similarity")
ax2.set_xlabel("Number of Shots in Prompt (k)")
ax2.set_ylabel("Average BLEU Score")
if bleu_results:
    shots = list(bleu_results.keys())
    scores = list(bleu_results.values())
    ax2.plot(shots, scores, marker='x', color='g')
    ax2.set_xticks(shots)
ax2.grid(True, linestyle='--', alpha=0.6)

plt.show()